In [1]:
import json
import pandas as pd
import Levenshtein as lv
import numpy as np
import os
import pickle
from tqdm import tqdm
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, LabelBinarizer
from sklearn_pandas import DataFrameMapper
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import classification_report, precision_recall_fscore_support, confusion_matrix
from sklearn.model_selection import cross_val_score

In [2]:
def measure_time_distance(timedelta):
    """
    Function that convert timedelta into seconds
    """
    distance = (timedelta.total_seconds())
    return distance

def measure_text_distance(x,y):
    """
    Function that calculate the Levenshtein distance
    between two submissions code.
    https://en.wikipedia.org/wiki/Levenshtein_distance
    """
    return lv.distance(x,y)

def calculate_distribution(df, column_name, exercise, mean=True):
    """
    df: Dataframe with submissions
    column_name: column with status

    Function to obtain distribution of submissions status
    return metrics and amount of submissions
    """
    total_amount_submissions = df.shape[0]
    submissions_grouped = df.groupby([column_name]).size()
    metrics = {}
    if mean:
        metrics = submissions_grouped/total_amount_submissions
    else:
        metrics = submissions_grouped
    metrics['exercise'] = str(exercise)
    metrics['submission_amount'] = total_amount_submissions
    return metrics

def shift_columns(df, shift_exercise=False):
    """
    Function that create 4 new columns
    shifted column contains submissions_content shifted to calculate the distance
    datetime_shifted contains datetime shifted to calculate the distance between times
    
    distance, Levenshtein distance between submissions
    time_dist, time distance between submissions in seconds
    """
    df.loc[:,'shifted'] = df['submissions_content'].shift().fillna(value='')
    df.loc[:,'guide.name.previous'] = df['guide.name'].shift().fillna(value='')
    df.loc[:,'datetime_shifted'] = df['datetime'].shift().fillna(value=pd.Timestamp(1800, 1, 1, 0))
    df.loc[:,'student.email.previous'] = df['student.email'].shift().fillna(value='')
    df['distance'] = df.apply(lambda x : measure_text_distance(x['submissions_content'],x['shifted']), axis=1)
    df['time_dist'] = df.apply(lambda x : measure_time_distance(x['datetime']-x['datetime_shifted']), axis=1)
    if shift_exercise:
        df.loc[:,'exercise.eid.previous'] = df['exercise.eid'].shift().fillna(value='')
        df.loc[:,'exercise.name.previous'] = df['exercise.name'].shift().fillna(value='')
        df.loc[:,'submissions_status.previous'] = df['submissions_status'].shift().fillna(value='')
        df.loc[:,'submissions_count.previous'] = df['submissions_submissions_count'].shift().fillna(value='')
        df['switch_exercise'] = df.apply(lambda x : x['exercise.name'] != x['exercise.name.previous'], axis=1)
    return df


In [30]:
dataset_home = "datasets/"
files = [
    #'2016-2c-introalgo-2c-2016.json', '2017-2c-introalgo-2c-2017.json',
    '2018-2c-introalgo-2018-2c-23Octubre.json',
]

for file in files:
    json_file = open(dataset_home+str(file), encoding='utf8')
    json_data = json.load(json_file)
    
    submissions = pd.io.json.json_normalize(
        json_data, 'submissions', [['guide','slug'], ['student','email'],['guide','name'], ['exercise','name'],['exercise','eid'],['guide','language','name']], record_prefix='submissions_', errors='ignore')
    submissions['datetime'] = pd.to_datetime(submissions['submissions_created_at'])
    submissions = submissions[~submissions['submissions_content'].isnull()]
    submissions = submissions[~(submissions['submissions_status']=='aborted')]
    #submissions = submissions[~(submissions['submissions_status']=='passed')]
    #submissions = submissions[~(submissions['submissions_status']=='passed_with_warnings')]
    submissions = submissions[submissions['guide.language.name'] == 'haskell']
    submissions = submissions.drop(
        submissions[(submissions['exercise.name'] == 'cifrasBinarias') | 
                    (submissions['exercise.name'] == 'ciclar') | 
                    (submissions['exercise.name'] == 'cifrasBase')].index)
    submissions = submissions.drop(
    submissions[(submissions['student.email'] == 'walteralini@gmail.com') |
                (submissions['student.email'] == 'romina.altamirano@gmail.com') |
                (submissions['student.email'] == 'kouichicruz@gmail.com')].index)
    submissions = submissions.drop(
        submissions[(submissions['student.email'] == 'ismaelpeker@gmail.com') &
                (submissions['submissions_created_at'].str.contains('2017'))].index)

In [31]:
calculate_distribution(submissions, "submissions_status", "todos", False)

submissions_status
errored                  7457
failed                   7855
passed                   3270
passed_with_warnings      790
exercise                todos
submission_amount       19372
dtype: object

In [32]:
exercises_attempted = 0
for student in submissions['student.email'].unique():
    exercises_attempted += submissions[submissions['student.email'] == student]['exercise.name'].nunique()
exercises_attempted

3043

In [33]:
submissions = submissions.sort_values(['student.email', 'datetime'])
submissions = shift_columns(submissions, True)

## Abandono por cambio de ejercicio

In [34]:
submissions[(submissions['switch_exercise']) 
            & ((submissions['submissions_status.previous'] == 'failed') | (submissions['submissions_status.previous'] == 'errored'))
           & (submissions['student.email'] == submissions['student.email.previous'])][
    ['exercise.name.previous', 'submissions_status.previous', 'time_dist',
     'switch_exercise', 'submissions_status', 'student.email.previous',
     'student.email', 'exercise.name', 'datetime','datetime_shifted']].iloc[1:]

,exercise.name.previous,submissions_status.previous,time_dist,switch_exercise,submissions_status,student.email.previous,student.email,exercise.name,datetime,datetime_shifted
0,Días,failed,650.573,True,errored,alecabralhillar@gmail.com,alecabralhillar@gmail.com,Guardas: Básico,2018-09-14 01:26:54.586,2018-09-14 01:16:04.013
16196,Orden de las Condiciones,failed,197199.229,True,errored,alecabralhillar@gmail.com,alecabralhillar@gmail.com,elementos3-upla,2018-09-20 22:43:59.399,2018-09-18 15:57:20.170
3743,cuentaBizarra,errored,303.834,True,failed,alecabralhillar@gmail.com,alecabralhillar@gmail.com,Orden de las Condiciones,2018-09-20 23:01:11.318,2018-09-20 22:56:07.484
1406,Orden de las Condiciones,failed,349.668,True,passed,alecabralhillar@gmail.com,alecabralhillar@gmail.com,No todo lo que brilla es Oro,2018-09-20 23:35:07.710,2018-09-20 23:29:18.042
4774,Variable Anónima,failed,402.478,True,failed,alecabralhillar@gmail.com,alecabralhillar@gmail.com,Pausa: Volvieron los tipos,2018-09-21 00:02:53.180,2018-09-20 23:56:10.702
3751,Pausa: Volvieron los tipos,failed,309492.188,True,failed,alecabralhillar@gmail.com,alecabralhillar@gmail.com,Orden de las Condiciones,2018-09-24 14:02:20.550,2018-09-21 00:04:08.362
6930,Puntos Para Setenta!,failed,450.074,True,failed,alecabralhillar@gmail.com,alecabralhillar@gmail.com,Tipando tuplas,2018-09-24 23:34:18.109,2018-09-24 23:26:48.035
3267,Tipando tuplas,failed,170335.717,True,failed,alecabralhillar@gmail.com,alecabralhillar@gmail.com,Pattern Matching con Tuplas,2018-09-26 23:15:25.313,2018-09-24 23:56:29.596
6326,Variable Anónima,failed,172.819,True,failed,alecabralhillar@gmail.com,alecabralhillar@gmail.com,Puntos Para Setenta!,2018-09-26 23:38:52.403,2018-09-26 23:35:59.584
487,Guardas: Básico,failed,141.764,True,failed,arnezdankevin@hotmail.com,arnezdankevin@hotmail.com,Otherwise,2018-08-23 23:40:44.972,2018-08-23 23:38:23.208


## Abandono por cambio de Sesion

In [35]:
threshold = 454
submissions[(submissions['switch_exercise'] == False) 
            & ((submissions['submissions_status.previous'] == 'failed') | (submissions['submissions_status.previous'] == 'errored'))
            & (submissions['time_dist'] > threshold)
            & (submissions['student.email'] == submissions['student.email.previous'])
          ][['exercise.name.previous', 'submissions_status.previous', 'time_dist',
     'switch_exercise', 'submissions_status', 'student.email.previous',
     'student.email', 'exercise.name', 'datetime']].iloc[1:]

,exercise.name.previous,submissions_status.previous,time_dist,switch_exercise,submissions_status,student.email.previous,student.email,exercise.name,datetime
4769,esMultiploDeTres,errored,495718.452,False,errored,alecabralhillar@gmail.com,alecabralhillar@gmail.com,esMultiploDeTres,2018-08-28 13:32:26.770
4770,esMultiploDeTres,errored,522.807,False,errored,alecabralhillar@gmail.com,alecabralhillar@gmail.com,esMultiploDeTres,2018-08-28 13:41:09.577
4772,esMultiploDeTres,errored,478.325,False,passed,alecabralhillar@gmail.com,alecabralhillar@gmail.com,esMultiploDeTres,2018-08-28 13:50:04.433
6010,esMultiploDe,failed,557.099,False,passed,alecabralhillar@gmail.com,alecabralhillar@gmail.com,esMultiploDe,2018-08-28 14:03:31.840
7544,esBisiesto,failed,798.163,False,errored,alecabralhillar@gmail.com,alecabralhillar@gmail.com,esBisiesto,2018-08-30 12:13:54.980
9351,haceFrioF,failed,572.483,False,errored,alecabralhillar@gmail.com,alecabralhillar@gmail.com,haceFrioF,2018-09-03 23:20:50.031
11839,Dispersión,errored,483362.533,False,passed,alecabralhillar@gmail.com,alecabralhillar@gmail.com,Dispersión,2018-09-13 12:20:54.522
12540,Pinos,errored,468.923,False,errored,alecabralhillar@gmail.com,alecabralhillar@gmail.com,Pinos,2018-09-13 13:43:36.564
13239,esCuadradoPerfecto,failed,32351.492,False,failed,alecabralhillar@gmail.com,alecabralhillar@gmail.com,esCuadradoPerfecto,2018-09-13 23:02:38.350
13735,Días,failed,838.731,False,failed,alecabralhillar@gmail.com,alecabralhillar@gmail.com,Días,2018-09-13 23:54:21.063


## Anoto el dataset
En la columna dropout_switch se anota con True los abandonos por cambio de ejercicio
En la columna dropout_session se anota con True los abandonos por session

In [36]:
submissions['dropout_switch'] = submissions.apply(
    lambda x: True if (
        (x['switch_exercise']) &
        ((x['submissions_status.previous'] == 'failed') | (x['submissions_status.previous'] == 'errored')) & 
        (x['student.email'] == x['student.email.previous'])
    ) else False, axis=1)

In [37]:
submissions['dropout_session'] = submissions.apply(
    lambda x: True if(
        (x['switch_exercise'] == False)
        & ((x['submissions_status.previous'] == 'failed') | (x['submissions_status.previous'] == 'errored'))
        & (x['time_dist'] > 454)
        & (x['student.email'] == x['student.email.previous'])
    ) else False, axis=1)

In [38]:
df = submissions
#df = df[df['student.email'] == df['student.email.previous']]

# Anotate dataset 2da forma

In [39]:
## Only for view
#submissions[submissions['dropout_switch']][['student.email', 'student.email.previous', 'switch_exercise', 'submissions_status.previous', 'exercise.name.previous', 'submissions_status', 'exercise.name', 'time_dist']]
#submissions[submissions['dropout_session']][['student.email', 'student.email.previous', 'switch_exercise', 'submissions_status.previous', 'exercise.name.previous', 'submissions_status', 'exercise.name', 'time_dist']]
test = pd.DataFrame()
df['dropout_session_tray'] = False
df['dropout_switch_tray'] = False
df = df.sort_values(['datetime_shifted', 'student.email.previous'], ascending=False)

for student in df['student.email.previous'].unique():
    for exercise_name in df[(df['student.email.previous'] == student) & (df['dropout_switch'] | df['dropout_session'])]['exercise.name.previous'].unique():
    #for exercise_name in df[(df['student.email.previous'] == student) & ((df['dropout_session']) | (df['dropout_switch']))]['exercise.name.previous'].unique()[:1]:
        exer_df = df[(df['exercise.name.previous'] == exercise_name) & (df['student.email.previous'] == student)]
        flag_session = False
        flag_switch = False
        for index, row in df[(df['exercise.name.previous'] == exercise_name) & (df['student.email.previous'] == student)].iterrows():
            if (flag_session and not(row['dropout_switch'])):
                #exer_df['dropout_session_tray'].iloc[index] = True
                df.at[index, 'dropout_session_tray'] = True
            if((row['dropout_session'] or flag_session) and not(row['dropout_switch'])):
                flag_session = True
                #exer_df['dropout_session_tray'].iloc[index] = True
                df.at[index, 'dropout_session_tray'] = True
            else:
                flag_session = False

            if (flag_switch and not(row['dropout_session'])):
                #exer_df['dropout_switch_tray'].iloc[index] = True
                df.at[index, 'dropout_switch_tray'] = True
            if ((row['dropout_switch'] or flag_switch) and not(row['dropout_session'])):
                flag_switch = True
                #exer_df['dropout_switch_tray'].iloc[index] = True
                df.at[index, 'dropout_switch_tray'] = True
            else:
                flag_switch = False
        test = test.append(exer_df, ignore_index=True)
#df[['switch_exercise', 'dropout_switch', 'dropout_session', 'dropout_session_tray', 'dropout_switch_tray']]

In [137]:
cant_soluciones = df.shape[0]
cant_switch = df[df['dropout_switch']].shape[0]
cant_session = df[df['dropout_session']].shape[0]

cant_switch_tray = df[df['dropout_switch_tray']].shape[0]
cant_session_tray = df[df['dropout_session_tray']].shape[0]

print("Cantidad submissions {}".format(cant_soluciones))

print("1era forma de anotar (solo la ultima submissions)")
print("Cantidad de dropout session {}. Proporcion respecto total submissions {:.3f}".format(cant_session, cant_session/cant_soluciones))
print("Cantidad de dropout switchs {}. Proporcion respecto total submissions {:.3f} ".format(cant_switch, cant_switch/cant_soluciones))
print("Considerando ambos tipos como el mismo cantidad {}  proporcion {:.3f}".format(cant_session + cant_switch,(cant_session + cant_switch)/cant_soluciones))

print("\n2da forma de anotar (anoto trayectoria)")
print('Cantidad de dropout session {}. Proporcion respecto total submissions {:.3f}'.format(cant_session_tray, cant_session_tray/cant_soluciones))
print('Cantidad de dropout switch {}. Proporcion respecto total submissions {:.3f}'.format(cant_switch_tray, cant_switch_tray/cant_soluciones))
print("Considerando ambos tipos como el mismo cantidad {}  proporcion {:.3f}".format(cant_session_tray + cant_switch_tray,(cant_session_tray + cant_switch_tray)/cant_soluciones))


#test[test['student.email.previous'] == 'elmaxisantillan@gmail.com'][['student.email.previous','datetime', 'datetime_shifted', 'exercise.name', 'submissions_status', 'exercise.name.previous','submissions_status.previous' ,'time_dist','switch_exercise', 'dropout_switch','dropout_switch_tray', 'dropout_session', 'dropout_session_tray']]

Cantidad submissions 19297
1era forma de anotar (solo la ultima submissions)
Cantidad de dropout session 644. Proporcion respecto total submissions 0.033
Cantidad de dropout switchs 1086. Proporcion respecto total submissions 0.056 
Considerando ambos tipos como el mismo cantidad 1730  proporcion 0.090

2da forma de anotar (anoto trayectoria)
Cantidad de dropout session 3382. Proporcion respecto total submissions 0.175
Cantidad de dropout switch 5781. Proporcion respecto total submissions 0.300
Considerando ambos tipos como el mismo cantidad 9163  proporcion 0.475


In [41]:
df.columns

Index(['submissions_content', 'submissions_created_at',
       'submissions_expectation_results', 'submissions_feedback',
       'submissions_result', 'submissions_sid', 'submissions_status',
       'submissions_submissions_count', 'submissions_test_results',
       'guide.slug', 'student.email', 'guide.name', 'exercise.name',
       'exercise.eid', 'guide.language.name', 'datetime', 'shifted',
       'guide.name.previous', 'datetime_shifted', 'student.email.previous',
       'distance', 'time_dist', 'exercise.eid.previous',
       'exercise.name.previous', 'submissions_status.previous',
       'submissions_count.previous', 'switch_exercise', 'dropout_switch',
       'dropout_session', 'dropout_session_tray', 'dropout_switch_tray'],
      dtype='object')

In [42]:
submissions_df = df[['shifted', 'guide.name.previous', 'datetime_shifted', 'student.email.previous',
    'distance', 'time_dist', 'exercise.eid.previous', 'exercise.name.previous',
    'submissions_status.previous', 'submissions_count.previous', 'switch_exercise',
    'dropout_switch', 'dropout_session', 'dropout_session_tray', 'dropout_switch_tray']]
submissions_df = submissions_df.rename(columns={'shifted':'content', 'guide.name.previous':'guide.name', 'datetime_shifted':'datetime', 'student.email.previous': 'student.email',
    'exercise.eid.previous' :'exercise.eid', 'exercise.name.previous': 'exercise.name',
    'submissions_status.previous':'submissions_status', 'submissions_count.previous':'submissions_count'})
submissions_df.to_pickle('introAlgo_final_anotado.pkl')

# Dropout Histogram 

In [207]:
dropout_switch_df = submissions_df.groupby(['student.email', 'dropout_switch_tray']).count().reset_index()
dropout_switch_df = dropout_switch_df[dropout_switch_df['dropout_switch_tray']].sort_values('content',ascending=False)[['student.email', 'content']]
dropout_switch_df.rename(columns={'content': 'dropout_switch_tray'}, inplace=True)
dropout_switch_df

,student.email,dropout_switch_tray
114,roccaedgardo@yahoo.com.ar,437
126,seba1999gonzalez@gmail.com,375
136,vale.pereyra27@gmail.com,282
69,knd-cramer@hotmail.com,228
7,casinga02@gmail.com,222
85,lxs_98@hotmail.com,195
13,cruzbelen62@gmail.com,183
49,isarivadero@hotmail.com,181
9,cba.carlos@gmail.com,179
66,julieta.pistoia@hotmail.com,176


In [161]:
dropout_session_df = submissions_df.groupby(['student.email', 'dropout_session_tray']).count().reset_index()
dropout_session_df = dropout_session_df[dropout_session_df['dropout_session_tray']].sort_values('content',ascending=False)[['student.email', 'content']]
dropout_session_df.rename(columns={'content': 'dropout_session_tray'}, inplace=True)
dropout_session_df

,student.email,dropout_session_tray
115,roccaedgardo@yahoo.com.ar,369
41,gonzalorivadero96@gmail.com,220
70,lautarobertone13@gmail.com,148
56,joaquinprotti@hotmail.com,114
33,frmarozzi_08@hotmail.com,107
100,nicosemiotica23@hotmail.com,105
7,casinga02@gmail.com,92
84,lxs_98@hotmail.com,88
65,julieta.pistoia@hotmail.com,88
121,rubentourn2000@gmail.com,87


In [211]:
dropouts_df = pd.merge(dropout_switch_df,dropout_session_df, on='student.email')
indexs = np.arange(0,401,20)[::-1]
indexs
#lst_dropouts_switch = [dropouts_df[dropouts_df['']]]

array([400, 380, 360, 340, 320, 300, 280, 260, 240, 220, 200, 180, 160,
       140, 120, 100,  80,  60,  40,  20,   0])

# Split dataset

In [8]:
base = 'datasets/introalgo/'
def train_dev_test(df, proportion=[.8,.9]):
    train_df = pd.DataFrame() 
    dev_df = pd.DataFrame() 
    test_df = pd.DataFrame() 
    exercises_names = df['exercise.name'].unique()
    for exercise in exercises_names:
        df_exer = df[df['exercise.name'] == exercise]
        train, dev  = np.split(df_exer.sample(frac=1), [int(.8*len(df_exer))])
        #print(train.shape, dev.shape, test.shape, exercise)
        train_df = train_df.append(train, ignore_index=True)
        dev_df = dev_df.append(dev, ignore_index=True)
        #test_df = test_df.append(test, ignore_index=True)
    return train_df, dev_df#, test_df

### Load dataframe from pickle

In [7]:
submissions = pd.read_pickle('dataframes_pkl/introAlgo_FINAL_anotado.pkl')
#submissions = submissions[~(submissions['student.email'] == 'ramiromariano.lerda@gmail.com')]

In [17]:
train, dev = train_dev_test(submissions[submissions['exercise.name'] == 'calcular'])
#train, dev, test = train_dev_test(submissions)
dev.shape

(63, 30)

In [62]:
train.to_pickle(base+'train801010.pkl')
dev.to_pickle(base+'dev801010.pkl')
test.to_pickle(base+'test801010.pkl')

## Load datasets from pickle

In [119]:
base = 'datasets/introalgo/'
train_df = pd.read_pickle(base+'train801010.pkl')
dev_df = pd.read_pickle(base+'dev801010.pkl')
test_df = pd.read_pickle(base+'test801010.pkl')

## Only for test i make new dataset and split this

In [3]:
#submissions = pd.read_pickle('dataframes_pkl/mumuki_io_FINAL_anotado.pkl')
submissions = pd.read_pickle('dataframes_pkl/introAlgo_FINAL_anotado.pkl')

# Expertise Dimension
## PSA and PCA calculate

In [620]:
submissions['PSA'] = 0
submissions['PCA'] = 0
students = submissions['student.email'].unique()
for student in tqdm(students, desc='students'):
    #calculo ejercicios terminados en verde para calcular promedio sin aplazos
    exercises_passed = submissions[(submissions['student.email'] == student) & (submissions['submissions_status'] == 'passed')]['exercise.name'].unique()
    #cantidad de ejercicios pasados
    amount_exer_passed = len(exercises_passed)
    #cantidad de soluciones enviadas por estudiante
    amount_submissions = submissions[submissions['student.email'] == student].shape[0]
    #cantidad de ejercicios intentados
    amount_exercises_attempted = submissions[submissions['student.email'] == student]['exercise.name'].nunique()
    cant_sol_acum = 0
    for exercise in exercises_passed:
        cant_soluciones = submissions[(submissions['student.email'] == student) & (submissions['exercise.name'] == exercise)].shape[0]
        cant_sol_acum += cant_soluciones
    if cant_sol_acum > 0:
        submissions.loc[submissions['student.email'] == student, ['PSA']] = amount_exer_passed / cant_sol_acum
    if amount_submissions > 0:
        submissions.loc[submissions['student.email'] == student, ['PCA']] = amount_exercises_attempted / amount_submissions




students:   0%|          | 0/75 [00:00<?, ?it/s]


students:   1%|▏         | 1/75 [00:00<00:14,  4.94it/s]


students:   4%|▍         | 3/75 [00:00<00:09,  7.28it/s]


students:   5%|▌         | 4/75 [00:00<00:11,  6.24it/s]


students:   7%|▋         | 5/75 [00:00<00:12,  5.66it/s]


students:   8%|▊         | 6/75 [00:01<00:11,  5.76it/s]


students:   9%|▉         | 7/75 [00:01<00:12,  5.43it/s]


students:  11%|█         | 8/75 [00:01<00:11,  5.60it/s]


students:  12%|█▏        | 9/75 [00:01<00:12,  5.32it/s]


students:  13%|█▎        | 10/75 [00:01<00:11,  5.43it/s]


students:  15%|█▍        | 11/75 [00:02<00:12,  5.31it/s]


students:  16%|█▌        | 12/75 [00:02<00:12,  5.25it/s]


students:  17%|█▋        | 13/75 [00:02<00:11,  5.41it/s]


students:  19%|█▊        | 14/75 [00:02<00:11,  5.32it/s]


students:  20%|██        | 15/75 [00:02<00:11,  5.22it/s]


students:  21%|██▏       | 16/75 [00:03<00:11,  5.16it/s]


students:  23%|██▎       | 17/75 [00:03<00:11,  5.11it

# PCA Mejorado

Ejercicios no pasados (Enp): $Enp = Ei - Ec$

Soluciones no exitosas (Sne): Sean $\left \{  s_{1}, .. ,s_{n}\right \}$ la cantidad soluciones enviadas por el mismo usuario para los ejercicios {{e_{1}, .. , e_{m}} cuya finalización no fue exitosa.

Promedio de error (Pde): $\frac{Enp} {\sum({s_1{}, .., s_{m}})} $
Luego de haber realizado estas definiciones podemos definir al promedio con aplazos mejorado de la siguiente forma
$$PCAM = PSA - (1 - Pde)$$



In [669]:
submissions['PCAM'] = 0
students = submissions['student.email'].unique()
for student in tqdm(students, desc='students'):
    psa_student = submissions[(submissions['student.email'] == student)]['PSA'].values[0]
    amount_submissions_not_passed = 0
    exercises_attempted = set(submissions[submissions['student.email'] == student]['exercise.name'].unique())
    exercises_passed = set(submissions[
        (submissions['student.email'] == student) & (submissions['submissions_status'] == 'passed')]['exercise.name'].unique())
    exercises_not_passed = list(exercises_attempted.difference(exercises_passed))
    for exercise in exercises_not_passed:
        amount_submissions_not_passed += submissions[
            (submissions['exercise.name'] == exercise) & (submissions['student.email'] == student)].shape[0]
    try:
        promedio_de_error = len(exercises_not_passed) / amount_submissions_not_passed
    except:
        promedio_de_error = 1
    pcam = (1 - promedio_de_error) - psa_student
    if pcam < 0:
        pcam = 0
    submissions.loc[submissions['student.email'] == student, ['PCAM']] = pcam





students:   0%|          | 0/75 [00:00<?, ?it/s]


students:   5%|▌         | 4/75 [00:00<00:01, 39.47it/s]


students:   9%|▉         | 7/75 [00:00<00:02, 33.24it/s]


students:  16%|█▌        | 12/75 [00:00<00:01, 37.59it/s]


students:  20%|██        | 15/75 [00:00<00:01, 33.18it/s]


students:  25%|██▌       | 19/75 [00:00<00:01, 33.17it/s]


students:  29%|██▉       | 22/75 [00:00<00:01, 31.98it/s]


students:  33%|███▎      | 25/75 [00:00<00:01, 30.75it/s]


students:  37%|███▋      | 28/75 [00:00<00:01, 30.32it/s]


students:  41%|████▏     | 31/75 [00:01<00:01, 29.93it/s]


students:  48%|████▊     | 36/75 [00:01<00:01, 30.76it/s]


students:  55%|█████▍    | 41/75 [00:01<00:01, 32.02it/s]


students:  61%|██████▏   | 46/75 [00:01<00:00, 32.89it/s]


students:  67%|██████▋   | 50/75 [00:01<00:00, 31.97it/s]


students:  73%|███████▎  | 55/75 [00:01<00:00, 32.83it/s]


students:  79%|███████▊  | 59/75 [00:01<00:00, 33.08it/s]


students:  84%|████████▍ | 63/75 [00:01<00:00, 3

# Dropout Dimension
## Dropout proportion / proporcion de abandonos

In [622]:
submissions['PA'] = 0
submissions['EA'] = 0
students = submissions['student.email'].unique()
for student in tqdm(students, desc='students'):
    #cantidad de soluciones marcadas como abandono
    dropout_amount = submissions[(submissions['student.email'] == student) & (submissions['dropout_tray'])].shape[0]
    #cantidad de soluciones del estudiante
    amount_submissions = submissions[(submissions['student.email'] == student)].shape[0]
    
    #cantidad de ejercicios abandonados
    exercises_dropped = submissions[(submissions['student.email'] == student) & (submissions['dropout'])]['exercise.name'].nunique()
    #cantidad de ejercicios intentados
    exercises_attempted = submissions[submissions['student.email'] == student]['exercise.name'].nunique()
    
    #Proporcion de ejercicios abandonados
    submissions.loc[submissions['student.email'] == student, ['EA']] = exercises_dropped / exercises_attempted
    
    #proporcion de soluciones marcadas como abandono
    submissions.loc[submissions['student.email'] == student, ['PA']] = dropout_amount / amount_submissions
    




students:   0%|          | 0/75 [00:00<?, ?it/s]


students:   7%|▋         | 5/75 [00:00<00:01, 44.61it/s]


students:  13%|█▎        | 10/75 [00:00<00:01, 45.41it/s]


students:  21%|██▏       | 16/75 [00:00<00:01, 47.55it/s]


students:  28%|██▊       | 21/75 [00:00<00:01, 47.83it/s]


students:  36%|███▌      | 27/75 [00:00<00:00, 49.11it/s]


students:  44%|████▍     | 33/75 [00:00<00:00, 49.91it/s]


students:  52%|█████▏    | 39/75 [00:00<00:00, 50.33it/s]


students:  60%|██████    | 45/75 [00:00<00:00, 50.91it/s]


students:  68%|██████▊   | 51/75 [00:00<00:00, 51.23it/s]


students:  76%|███████▌  | 57/75 [00:01<00:00, 51.66it/s]


students:  84%|████████▍ | 63/75 [00:01<00:00, 51.18it/s]


students:  92%|█████████▏| 69/75 [00:01<00:00, 51.70it/s]


students: 100%|██████████| 75/75 [00:01<00:00, 51.98it/s]




### Proporcion Ponderada de Abandono (PPA)
Sean $\left \{  e_{1}, .. ,e_{n}\right \}$ los ejercicios abandonados por un estudiante x, es decir aquellos ejercicios con al menos una solución marcada como abandono. Sean $\left \{a_{1}e_{1},.,a_{m}e_{n}\right \}$ la cantidad de soluciones consideradas como abandono por ejercicio y sean $\left \{s_{1}e_{1},.., s_{n}e_{m}\right \}$ la cantidad total de soluciones por ejercicio. Definimos esta métrica como: 
$$ \frac {\sum(\frac{a_{1}e_{1}} {s_{1}e_{1}},..,\frac{a_{n}e_{n}} {s_{n}e_{n}})} {n} $$


In [623]:
submissions['PPA'] = 0
students = submissions['student.email'].unique()
for student in tqdm(students):
    exercises_dropped_out = submissions[(submissions['dropout']) & (submissions['student.email'] == student)]['exercise.name'].unique()
    amount_exercises_dropped_out = len(exercises_dropped_out)
    acumulator = 0
    for exercise in exercises_dropped_out:
        submissions_amount_exer = submissions[(submissions['exercise.name'] == exercise) & (submissions['student.email'] == student)].shape[0]
        submission_dropped = submissions[(submissions['exercise.name'] == exercise) & (submissions['student.email'] == student) & (submissions['dropout_tray'])].shape[0]
        acumulator += submission_dropped / submissions_amount_exer
    if amount_exercises_dropped_out == 0:
        wn = 0
    else:
        wn = acumulator / amount_exercises_dropped_out
    submissions.loc[submissions['student.email'] == student, ['PPA']] = wn




  0%|          | 0/75 [00:00<?, ?it/s]


  3%|▎         | 2/75 [00:00<00:06, 11.23it/s]


  5%|▌         | 4/75 [00:00<00:06, 10.39it/s]


  8%|▊         | 6/75 [00:00<00:07,  9.64it/s]


 11%|█         | 8/75 [00:00<00:06, 10.36it/s]


 12%|█▏        | 9/75 [00:00<00:06,  9.77it/s]


 15%|█▍        | 11/75 [00:01<00:06,  9.67it/s]


 16%|█▌        | 12/75 [00:01<00:06,  9.44it/s]


 19%|█▊        | 14/75 [00:01<00:06,  9.32it/s]


 20%|██        | 15/75 [00:01<00:06,  8.83it/s]


 23%|██▎       | 17/75 [00:01<00:06,  9.13it/s]


 24%|██▍       | 18/75 [00:01<00:06,  9.03it/s]


 25%|██▌       | 19/75 [00:02<00:06,  9.01it/s]


 28%|██▊       | 21/75 [00:02<00:06,  9.00it/s]


 29%|██▉       | 22/75 [00:02<00:06,  8.80it/s]


 31%|███       | 23/75 [00:02<00:06,  8.48it/s]


 32%|███▏      | 24/75 [00:02<00:05,  8.50it/s]


 33%|███▎      | 25/75 [00:02<00:05,  8.50it/s]


 35%|███▍      | 26/75 [00:03<00:06,  8.13it/s]


 36%|███▌      | 27/75 [00:03<00:06,  7.89it/s]


 37%|███▋  

# Nivel Intensidad
- Promedio de tiempo entre soluciones (PTT)
- Promedio de distancia de edición entre soluciones (PDL)

In [624]:
submissions['PTT'] = 0
submissions['PDL'] = 0
students = submissions['student.email'].unique()
for student in tqdm(students):
    ptt = submissions[(submissions['student.email'] == student) &
            (submissions['time_dist'] <= 454) & (submissions['time_dist'] >0)]['time_dist'].mean()
    pdl = submissions[(submissions['student.email'] == student) & (submissions['switch_exercise'] == False)]['distance'].mean()
    submissions.loc[submissions['student.email'] == student, ['PTT']] = ptt
    submissions.loc[submissions['student.email'] == student, ['PDL']] = pdl




  0%|          | 0/75 [00:00<?, ?it/s]


  8%|▊         | 6/75 [00:00<00:01, 55.40it/s]


 16%|█▌        | 12/75 [00:00<00:01, 57.35it/s]


 25%|██▌       | 19/75 [00:00<00:00, 59.36it/s]


 35%|███▍      | 26/75 [00:00<00:00, 60.86it/s]


 44%|████▍     | 33/75 [00:00<00:00, 62.53it/s]


 52%|█████▏    | 39/75 [00:00<00:00, 61.38it/s]


 61%|██████▏   | 46/75 [00:00<00:00, 62.37it/s]


 71%|███████   | 53/75 [00:00<00:00, 62.89it/s]


 80%|████████  | 60/75 [00:00<00:00, 63.42it/s]


 89%|████████▉ | 67/75 [00:01<00:00, 63.66it/s]


 99%|█████████▊| 74/75 [00:01<00:00, 63.92it/s]


100%|██████████| 75/75 [00:01<00:00, 63.35it/s]

# Insistencia ponderada por abandono (IPA)
Con el objetivo de intentar capturar cuán insistente es el estudiante x cada vez que comienza a resolver un ejercicio construimos la siguiente característica. Sean  $\left \{  e_{1}, .. ,e_{n}\right \}$ los ejercicios que intento resolver un estudiante x. Sean,  $\left \{ t_{1}, .. ,t_{m}\right \}$ la cantidad de veces que hayan terminando en estado de finalización (exitoso o abandono) para cada ejercicio. Sean  $\left \{  s_{1}, .. ,s_{l}\right \}$ la cantidad de soluciones realizadas por cada vez que intento resolverlo llegando a ese estado de finalización por ejercicio. Definimos nuestra métrica como:
$$ \frac{\sum{\frac{t_{1}}{s_{1}} .... \frac{t_{m}} {s_{l}}}}{n} $$ 

In [625]:
submissions['IPA'] = 0
students = submissions['student.email'].unique()
for student in tqdm(students):
    exercises_attempted = submissions[(submissions['student.email'] == student)]['exercise.name'].unique()
    acumulator = 0
    for exercise in exercises_attempted:
        exer_df = submissions[(submissions['student.email'] == student) & (submissions['exercise.name'] == exercise)]
        amount_dropouts = exer_df[exer_df['dropout']].shape[0]
        passed_amount = exer_df[exer_df['submissions_status'] == 'passed'].shape[0]
        if passed_amount > 0:
            total_attempt = amount_dropouts + 1
        else:
            total_attempt = amount_dropouts
        submissions_by_exer = exer_df.shape[0]
        acumulator += total_attempt / submissions_by_exer
        #print('student {} exercise {}'.format(student, exercise))
        #print('attempts {} submission by exer {} acumulator {} \n'.format(total_attempt, submissions_by_exer, acumulator))
    wn = acumulator / len(exercises_attempted)
    #print('================= {} ============='.format(wn))
    submissions.loc[submissions['student.email'] == student, ['IPA']] = wn
    




  0%|          | 0/75 [00:00<?, ?it/s]


  1%|▏         | 1/75 [00:00<00:20,  3.67it/s]


  3%|▎         | 2/75 [00:00<00:15,  4.82it/s]


  4%|▍         | 3/75 [00:00<00:13,  5.38it/s]


  5%|▌         | 4/75 [00:00<00:15,  4.57it/s]


  7%|▋         | 5/75 [00:01<00:16,  4.28it/s]


  8%|▊         | 6/75 [00:01<00:16,  4.24it/s]


  9%|▉         | 7/75 [00:01<00:17,  3.96it/s]


 11%|█         | 8/75 [00:01<00:16,  4.11it/s]


 12%|█▏        | 9/75 [00:02<00:16,  3.96it/s]


 13%|█▎        | 10/75 [00:02<00:16,  3.97it/s]


 15%|█▍        | 11/75 [00:02<00:16,  3.86it/s]


 16%|█▌        | 12/75 [00:03<00:16,  3.84it/s]


 17%|█▋        | 13/75 [00:03<00:15,  3.98it/s]


 19%|█▊        | 14/75 [00:03<00:15,  3.92it/s]


 20%|██        | 15/75 [00:03<00:15,  3.89it/s]


 21%|██▏       | 16/75 [00:04<00:15,  3.88it/s]


 23%|██▎       | 17/75 [00:04<00:15,  3.86it/s]


 24%|██▍       | 18/75 [00:04<00:14,  3.83it/s]


 25%|██▌       | 19/75 [00:04<00:14,  3.88it/s]


 27%|██▋       

# Dimension Ejercicio
## Dificultad

### Promedio de cantidad soluciones para aprobar(PCSA)

In [626]:
submissions['PCSA'] = 0
exercises = submissions['exercise.name'].unique()
for exercise in exercises:
    submissions.loc[submissions['exercise.name'] == exercise, ['PCSA']] = submissions[
        (submissions['exercise.name'] == exercise) & (submissions['submissions_status'] == 'passed')]['submissions_count'].mean()

### Cantidad de abandonos por ejercicio (APE)

In [627]:
submissions['APE'] = 0
exercises = submissions['exercise.name'].unique()
for exercise in exercises:
    exer_df = submissions[submissions['exercise.name'] == exercise]
    submissions_amount = exer_df.shape[0]
    dropout_amount = exer_df[exer_df['dropout_tray']].shape[0]
    submissions.loc[submissions['exercise.name'] == exercise, ['APE']] = dropout_amount/submissions_amount

### Cantidad de Abandonos por estudiante (CAPE)
Cantidad de Abandonos por estudiante: número de estudiantes que abandonaron el ejercicio / cantidad de soluciones marcadas como abandono.

In [628]:
submissions['CAPE'] = 0
exercises = submissions['exercise.name'].unique()
for exercise in exercises:
    exer_df = submissions[submissions['exercise.name'] == exercise]
    students_attempted = exer_df['student.email'].unique()
    dropout_amount = exer_df['dropout_tray'].shape[0]
    if dropout_amount > 0:
        cape = len(students_attempted) / dropout_amount
    else:
        cape = 0
    submissions.loc[submissions['exercise.name'] == exercise, ['CAPE']] = cape

### Completitud (COMP)

In [629]:
submissions['COMP'] = 0
exercises = submissions['exercise.name'].unique()
for exercise in exercises:
    exer_df = submissions[submissions['exercise.name'] == exercise]
    students_attempted = exer_df['student.email'].nunique()
    passed_submissions = exer_df[exer_df['submissions_status'] == 'passed'].shape[0]
    submissions.loc[submissions['exercise.name'] == exercise, ['COMP']] = passed_submissions / students_attempted

## Train set

In [925]:
submissions['length'] = 0
submissions['length'] = submissions.apply(lambda x: len(x['content']), axis=1)

In [930]:
submissions.sort_values('length', ascending=False)['length']
#print(submissions[submissions['length'] == 586]['content'][13957])

13957    586
20981    518
20116    514
13958    509
13951    508
13949    508
13953    507
13955    507
13954    507
13956    507
607      499
13973    499
20115    489
10734    471
12311    470
10733    467
16500    467
10732    465
17581    464
17582    464
10731    463
21109    460
17583    460
1340     459
13963    455
19923    453
17677    451
12310    448
20114    448
20113    446
        ... 
20347      8
4565       8
15966      8
18148      8
4605       7
5548       7
5549       7
14997      7
417        7
8057       6
17558      6
8056       6
18152      6
10184      5
20215      5
10166      5
10201      4
5293       4
20214      4
10200      3
10189      2
20213      2
6628       1
3428       1
2721       1
2720       1
2719       1
3427       1
2725       1
2723       1
Name: length, Length: 19371, dtype: int64

In [61]:
submissions = pd.read_pickle('dataframes_pkl/mumuki_io_FINAL_anotado.pkl')
#submissions = pd.read_pickle('dataframes_pkl/introAlgo_FINAL_anotado.pkl')
submissions['PTT'].fillna(value=0, inplace=True)
submissions['PDL'].fillna(value=0, inplace=True)



In [62]:
submissions.columns

Index(['content', 'guide.name', 'datetime', 'student.email', 'distance',
       'time_dist', 'exercise.eid', 'exercise.name', 'submissions_status',
       'submissions_count', 'switch_exercise', 'dropout_switch',
       'dropout_session', 'dropout_session_tray', 'dropout_switch_tray',
       'dropout', 'dropout_tray', 'PSA', 'PCA', 'PCAM', 'PA', 'EA', 'PPA',
       'PTT', 'PDL', 'IPA', 'PCSA', 'APE', 'CAPE', 'COMP'],
      dtype='object')

In [63]:
train_df, test_df= train_dev_test(submissions)

In [64]:
features = ['content', 'submissions_count','time_dist', 'distance', 'PA', 'EA','PCA','PSA','PCAM', 'PPA', 'PTT','PDL', 'IPA', 'PCSA', 'APE', 'CAPE', 'COMP','exercise.eid']
target = 'dropout_tray'
X_train = train_df[features]
Y_train = train_df[target]

In [65]:
X_test = test_df[features]
Y_test = test_df[target] 

## dev Set

# Probando con DataFrameMapper

In [66]:
mapper = DataFrameMapper([
    #('content', CountVectorizer()),
    (['submissions_count'], None),
    (['PA','APE'], None),
 ])
mapper.fit_transform(X_train)
pipe2 = Pipeline([
    ('mapper', mapper),
    ('classifier', LogisticRegression(C=1,penalty='l2', solver='liblinear',tol=1e-6, class_weight='balanced',
                         max_iter=int(1e6), warm_start=True, verbose=3))
])

from timeit import default_timer as timer

start = timer()
pipe2.fit(X_train, Y_train)
end = timer()
print(end - start)


[LibLinear]0.592233096002019


In [67]:
from timeit import default_timer as timer

start = timer()
preds = pipe2.predict(X_test)
end = timer()
print(end - start) # Time in seconds, e.g. 5.38091952400282

report = classification_report(Y_test, preds, digits=4)
cm = confusion_matrix(Y_test, preds)
print(report)
p, r, f1, s = precision_recall_fscore_support(preds, Y_test, average='weighted')
print(p, r, f1, s)

0.014533468005538452
              precision    recall  f1-score   support

       False     0.8809    0.7605    0.8163     32906
        True     0.5799    0.7628    0.6589     14262

   micro avg     0.7612    0.7612    0.7612     47168
   macro avg     0.7304    0.7616    0.7376     47168
weighted avg     0.7899    0.7612    0.7687     47168

0.7613947865473739 0.761172829036635 0.7536663964566175 None


In [68]:
X_test.shape

(47168, 18)

In [69]:
X_all = submissions[features]
Y_all = submissions[target]

In [70]:
from sklearn import metrics
scores = cross_val_score(pipe2, X_all, Y_all, cv=10, n_jobs=-1,  scoring='f1_weighted')
#scores_test = cross_val_score(pipe_dummy, X_all, Y_all, cv=10, scoring='f1_weighted')
print("10-fold-cross-validation {}".format(np.array(scores).mean()))
#print("10-fodl-cross-validation {}".format(np.array(scores_test).mean()))

10-fold-cross-validation 0.767003578744874


In [3]:
from sklearn.dummy import DummyClassifier

In [20]:
pipe_dummy = Pipeline([
    ('mapper', mapper),
    ('classifier', DummyClassifier())
])


In [21]:
pipe_dummy.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('mapper', DataFrameMapper(default=False, df_out=False,
        features=[(['submissions_count'], None)], input_df=False,
        sparse=False)), ('classifier', DummyClassifier(constant=None, random_state=None, strategy='stratified'))])

In [18]:
preds = pipe_dummy.predict(X_test)
p, r, f1, s = precision_recall_fscore_support(preds, Y_test, average='weighted')
print(p, r, f1, s)
print(classification_report(Y_test, preds, digits=4))

NameError: name 'pipe_dummy' is not defined

In [1054]:
submissions[submissions['dropout_tray']].shape[0] / submissions.shape[0]

0.4730266893810335